In [1]:
import tensorflow as tf 
import os 
from tensorflow import keras 
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras import Sequential
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential, load_model

In [2]:
count =0 
dir =os.listdir('train/')
for dir in dir:
    files = list(os.listdir('train/'+dir))
    print(dir + 'folder has '+ str(len(files))+"train")
    count = count + len(files)
print("image folder has "+ str(count) + "image")    

daisyfolder has 501train
dandelionfolder has 646train
rosefolder has 497train
sunflowerfolder has 495train
tulipfolder has 607train
image folder has 2746image


In [3]:
base_dir = ("train/")

In [4]:
train_dir = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed=123,
    image_size = (180, 180),
    subset="training",
    validation_split=0.2,
    batch_size=32  
)

Found 2746 files belonging to 5 classes.
Using 2197 files for training.


In [5]:
val_dir = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed=123,
    image_size = (180, 180),
    subset="validation",
    validation_split=0.2,
    batch_size=32 
)

Found 2746 files belonging to 5 classes.
Using 549 files for validation.


In [6]:
flower_name = train_dir.class_names

In [7]:
autotune = tf.data.AUTOTUNE

In [8]:
train_dir = train_dir.cache().shuffle(1000).prefetch(buffer_size = autotune)
val_dir = val_dir.cache().prefetch(buffer_size = autotune)

In [9]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal", input_shape = (180,180, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

c:\Users\abhis\AppData\Local\anaconda3\envs\brest\lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3,padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64 , 3 , padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.2),
    Dense(128, activation="relu"),
    Dense(5)        
])

In [11]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"]
              )

In [12]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 180, 180, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 90, 90, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 90, 90, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 45, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,989,285 (15.22 MB)

 Trainable params: 3,989,285 (15.22 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
#history = model.fit(train_dir, epochs=15, validation_data=val_dir)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,         
    restore_best_weights=True  
)

history = model.fit(
    train_dir,
    epochs=15,
    validation_data=val_dir,
    callbacks=[early_stopping]  
)


Epoch 1/15


69/69 ━━━━━━━━━━━━━━━━━━━━ 55s 691ms/step - accuracy: 0.3456 - loss: 1.5370 - val_accuracy: 0.5264 - val_loss: 1.2692
Epoch 2/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 50s 715ms/step - accuracy: 0.5637 - loss: 1.0815 - val_accuracy: 0.5719 - val_loss: 1.1760
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 52s 758ms/step - accuracy: 0.6290 - loss: 0.9566 - val_accuracy: 0.6120 - val_loss: 1.0303
Epoch 4/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 41s 595ms/step - accuracy: 0.6660 - loss: 0.8682 - val_accuracy: 0.6284 - val_loss: 1.0673
Epoch 5/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 46s 665ms/step - accuracy: 0.6725 - loss: 0.8284 - val_accuracy: 0.6758 - val_loss: 0.8457
Epoch 6/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 42s 600ms/step - accuracy: 0.6919 - loss: 0.7861 - val_accuracy: 0.6266 - val_loss: 0.9760
Epoch 7/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 47s 690ms/step - accuracy: 0.6965 - loss: 0.7496 - val_accuracy: 0.6940 - val_loss: 0.8284
Epoch 8/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 71s 520ms/step - accuracy: 0.7517 - loss: 0.6438 - val_accuracy: 0.695

In [ ]:
input_image = tf.keras.utils.load_img( r"C:\Users\abhis\OneDrive\Desktop\Deep_Learning\sample\daisy.webp")
input_image_array = tf.keras.utils.img_to_array(input_image)
